In [41]:
import cv2
import os
from tqdm import tqdm
import numpy as np

# 预设函数

In [42]:
def save_tempimg(image):
    '''
    先清空当前文件夹，然后保存图片到temp_img文件夹下
    input: 
        a cv2 format image
    output:
        None (save image to temp_img/)
    '''
    all_files = os.listdir("temp_img/") # 获取目录下所有文件和目录

    # 遍历所有文件和子目录
    for file_or_dir in all_files:
        # 构建完整的文件或目录路径
        file_or_dir_path = os.path.join("temp_img/", file_or_dir)
        
        # 检查是否是文件并删除
        if os.path.isfile(file_or_dir_path):
            os.remove(file_or_dir_path) # 删除temp_img文件夹下的所有文件

    cv2.imwrite('temp_img/temp.png', image) # 保存 temp.png

def save_tempimg2(image):
    '''
    先清空当前文件夹，然后保存temp2图片到temp_img文件夹下
    input: 
        a cv2 format image
    output:
        None (save image to temp_img/)
    '''
    cv2.imwrite('temp_img/temp2.png', image)

def save_tempimg3(image):
    '''
    先清空当前文件夹，然后保存temp3图片到temp_img文件夹下
    input: 
        a cv2 format image
    output:
        None (save image to temp_img/)
    '''
    cv2.imwrite('temp_img/temp3.png', image)

# def generate_padding(image, border_size=1):
#     '''
#     老版for loop brightness difference processing所使用的函数
#     为图像添加padding
#     input: 
#         image: a cv2 format image
#         border_size: int, padding size
#     output:
#         padded_image: a cv2 format image
#     '''
#     # Get the dimensions of the original image
#     height, width = image.shape

#     # Create a new image with the desired size
#     padded_image = np.zeros((height + 2 * border_size, width + 2 * border_size), dtype=np.uint8)

#     # Copy the original image to the center of the new image
#     padded_image[border_size:border_size+height, border_size:border_size+width] = image

#     return padded_image

- 读取图片

In [43]:
from IPython.display import Image, display

image = cv2.imread('4.jpg', cv2.IMREAD_GRAYSCALE) #在这里更改图片路径

# Brightness Difference Processing (第一部分)

In [44]:
# 老版for loop代码
# brightness_color_diff = image.copy()
# image_with_padding = generate_padding(image)

# threshold_difference = 4
# for i in tqdm(range(len(brightness_color_diff))): # for each row
#     for j in range(len(brightness_color_diff[i])): # for each column
#         current = int(image_with_padding[i][j])
#         top = int(image_with_padding[i-1][j])
#         bot = int(image_with_padding[i+1][j])
#         left = int(image_with_padding[i][j-1])
#         right = int(image_with_padding[i][j+1])
#         top_diff = abs(current - top)
#         bot_diff = abs(current - bot)
#         left_diff = abs(current - left)
#         right_diff = abs(current - right)
#         diff = max(top_diff, bot_diff, left_diff, right_diff)
#         if diff > threshold_difference or brightness_color_diff[i][j] <= 150:
#             brightness_color_diff[i][j] = 0 
#         if brightness_color_diff[i][j] > 150:
#             brightness_color_diff[i][j] = 255

In [45]:
def generate_padding(image, pad_width=1, pad_value=0):
    """为图像生成周围的填充（padding）。"""
    return np.pad(image, ((pad_width, pad_width), (pad_width, pad_width)), 'constant', constant_values=(pad_value,))

# 假设 `image` 是一个灰度图像
image = cv2.imread('3.jpg', cv2.IMREAD_GRAYSCALE)
image_with_padding = generate_padding(image)

image = image.astype(np.int16)
image_with_padding = image_with_padding.astype(np.int16)

# 使用向量化方式进行处理
threshold_difference = 4
# 计算相邻像素之间的差异
top_diff = np.abs(image_with_padding[:-2, 1:-1] - image_with_padding[1:-1, 1:-1]) # [:-2, 1:-1]表示去除了最后两行和第一列+最后一列的所有行和列, [1:-1, 1:-1]表示去除了第一行+最后一行和第一列+最后一列的所有行和列
bot_diff = np.abs(image_with_padding[2:, 1:-1] - image_with_padding[1:-1, 1:-1]) # [2:, 1:-1]表示去除了前两行和第一列+最后一列的所有行和列, [1:-1, 1:-1]表示去除了第一行+最后一行和第一列+最后一列的所有行和列
left_diff = np.abs(image_with_padding[1:-1, :-2] - image_with_padding[1:-1, 1:-1]) # [1:-1, :-2]表示去除了第一行+最后一行和最后两列的所有行和列, [1:-1, 1:-1]表示去除了第一行+最后一行和第一列+最后一列的所有行和列
right_diff = np.abs(image_with_padding[1:-1, 2:] - image_with_padding[1:-1, 1:-1])  # [1:-1, 2:]表示去除了第一行+最后一行和前两列的所有行和列, [1:-1, 1:-1]表示去除了第一行+最后一行和第一列+最后一列的所有行和列

# 计算最大差异
# max_diff = np.maximum(np.maximum(top_diff, bot_diff), np.maximum(left_diff, right_diff))
max_diff = np.maximum(np.maximum(np.maximum(top_diff, bot_diff), left_diff), right_diff)

# 应用阈值条件
brightness_color_diff = image.copy()
brightness_color_diff[image > 150] = 255
brightness_color_diff[image <= 150] = 0
brightness_color_diff[max_diff > threshold_difference] = 0

image = image.astype(np.int8)
image_with_padding = image_with_padding.astype(np.int8)

- 所有处理过的图片都会保存在temp_img/ 文件夹中，如果没有此文件夹，请先在根目录下建立一个名为"temp_img"的文件夹

In [46]:
save_tempimg(brightness_color_diff) # 保存图片到temp_img文件夹下

# Denoising (第二部分)

- 本部分一共尝试过三种算法
1. intergral image
2. intergral image vectorized
3. 2D Convolution
- PPT中展示的结果是第一种算法（integral image）得到的结果 耗时约为10s
- 第二种算法没有完全debug，虽然计算速度非常快（<1s），但是得到的结果和第一种算法不一致
- Convolution算法并没有完全适配，可能会出现长达几十分钟的运行时间，不推荐使用

## Integral Image

In [48]:
import numpy as np
from tqdm import tqdm

def calculate_integral_image(image):
    integral_image = np.cumsum(np.cumsum(image, axis=0), axis=1)
    return integral_image

def fast_thresholding(image, kernel_n, threshold):
    # Calculate the integral image
    integral_image = calculate_integral_image(image)

    # Create an output image
    output_image = np.ones_like(image) * 255
    threshold_value = (kernel_n**2) * threshold

    # Calculate border size
    border_size = kernel_n // 2

    # Iterate over each pixel in the original image
    for y in range(border_size, image.shape[0] - border_size):
        for x in range(border_size, image.shape[1] - border_size):
            # Calculate the sum of the kernel area using the integral image
            total = integral_image[y + border_size, x + border_size]
            total -= integral_image[y + border_size, x - border_size - 1]
            total -= integral_image[y - border_size - 1, x + border_size]
            total += integral_image[y - border_size - 1, x - border_size - 1]
            # Calculate the number of black pixels (assuming black is 0 and white is 255)
            black_pixel_count = (kernel_n**2) - total / 255
            # Apply threshold
            if black_pixel_count > threshold_value:
                output_image[y, x] = 0

    return output_image

# Applying the optimized thresholding function
output = fast_thresholding(brightness_color_diff, kernel_n=9, threshold=0.2)


In [49]:
save_tempimg2(output) # 保存图片到temp_img文件夹下

## Integral Image Vectorized

In [50]:
# import numpy as np

# def fast_thresholding_vectorized(image, kernel_n=9, threshold=0.2):
#     # Calculate the integral image
#     integral_image = np.cumsum(np.cumsum(image, axis=0), axis=1)
    
#     # Expand the integral image by padding to handle borders
#     padded_integral = np.pad(integral_image, ((kernel_n//2+1,), (kernel_n//2+1,)), 'constant', constant_values=0)
    
#     # Calculate sums over the kernel for each pixel in a vectorized manner
#     total = (
#         padded_integral[kernel_n:, kernel_n:] - 
#         padded_integral[:-kernel_n, kernel_n:] - 
#         padded_integral[kernel_n:, :-kernel_n] + 
#         padded_integral[:-kernel_n, :-kernel_n]
#     )
    
#     # Calculate the average intensity within the kernel area for each pixel
#     avg_intensity = total / (kernel_n ** 2)
    
#     # Create an output image based on the threshold
#     # Assume black is 0 and white is 255
#     output_image = np.where(avg_intensity <= threshold * 255, 0, 255).astype(np.uint8)
    
#     # Adjust the output image size to match the input image size
#     output_image = output_image[(kernel_n//2):-((kernel_n-1)//2), (kernel_n//2):-((kernel_n-1)//2)]
    
#     return output_image

In [51]:
# denoised_img = fast_thresholding_vectorized(brightness_color_diff, kernel_n=9, threshold=0.2)

In [52]:
# save_tempimg3(denoised_img)

## 2D Convolution

In [ ]:
# from scipy.signal import convolve2d
# kernel_n = 9 # must be odd
# threshold = 0.4                             # Adjustable threshold value
# kernel = np.ones((kernel_n, kernel_n))
# result_array = convolve2d(brightness_color_diff, kernel, mode='same', boundary='fill', fillvalue=0)
# # multiply result_array by 1/225
# white_pixel_count = result_array / 255
# black_pixel_ratio = (81 - white_pixel_count) / 81

In [ ]:
# img_convolution_test = brightness_color_diff.copy()
# for i in tqdm(range(len(img_convolution_test))):
#     for j in range(len(img_convolution_test[i])):
#         if black_pixel_ratio[i][j] < threshold:
#             img_convolution_test[i][j] = 255
#         else:
#             img_convolution_test[i][j] = 0

In [ ]:
# save_tempimg2(img_convolution_test)

# Uniform Region Detection

In [10]:
def uniform_region_detection(image, kernel_n):
    # Calculate the integral image
    integral_image = calculate_integral_image(image)

    # Create an output image
    output_image = image.copy()

    # Calculate border size
    border_size = kernel_n // 2

    # Iterate over each pixel in the original image
    for y in range(border_size, image.shape[0] - border_size):
        for x in range(border_size, image.shape[1] - border_size):
            # Calculate the sum of the kernel area using the integral image
            total = integral_image[y + border_size, x + border_size]
            total -= integral_image[y + border_size, x - border_size - 1]
            total -= integral_image[y - border_size - 1, x + border_size]
            total += integral_image[y - border_size - 1, x - border_size - 1]
            # Calculate the number of black pixels (assuming black is 0 and white is 255)
            black_pixel_count = (kernel_n**2) - total / 255
            # Apply threshold
            if black_pixel_count == 0:
                output_image[y, x] = 170 # gray color for region able for processing

    return output_image

image_uniform = uniform_region_detection(output, kernel_n=301)

In [11]:
save_tempimg3(image_uniform)

# Test point calculation

- 本部分是花费最多时间的地方，需要更多的优化
- 给出是否能进行试验的结论，如果可以给出用于测试的位置

In [55]:
# Check if 170 in the image
def check_170(image):
    for i in range(len(image)):
        for j in range(len(image[i])):
            if image[i][j] == 170:
                return True
    return False

True


In [12]:
if check_170(image_uniform) == True:
    max_distance = 0
    max_cordinate = (0, 0)
    for i in range(len(image_uniform)):
        for j in range(len(image_uniform[i])):
            if image_uniform[i][j] == 170:
                distance = 0
                try: # Avoid index out of range
                    if image_uniform[i+max_distance][j] and image_uniform[i-max_distance][j] and image_uniform[i][j+max_distance] and image_uniform[i][j-max_distance] == 170:
                        max_cordinate = (i, j)
                        while image_uniform[i+distance][j] and image_uniform[i-distance][j] and image_uniform[i][j+distance] and image_uniform[i][j-distance] == 170:
                            distance += 1
                            max_distance = distance
                    else:
                        continue
                except:
                    continue
    print("The pixel to conduct the processing is: ", max_cordinate, "with the distance of: ", max_distance)
else:
    print("No pixel to conduct the processing")